In [7]:
# Fashion MNIST դատասեթի վրա կիրառել ենք PCA և լուծել ենք կլասիֆիկացիայի խնդիր,
# որպեսզի տեսնենք դրա ազդեցությունը ժամանակի և ինֆորմացիայի որակի վրա

import numpy as np
import time
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import fashion_mnist
import random
import os

random.seed(53)
np.random.seed(53)
os.environ['PYTHONHASHSEED'] = '53'
# Ներբեռնել ենք տվյալները
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
# Այստեղ 28*28 չափի նկարները դարձրել ենք 784*1 չափի
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)

# Տվյալներից հեռացրել ենք միջինը և բաժանել ենք ստանդարտ շեղման վրա,որը ընդունված տեխնիկա է
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Կիրառել ենք PCA
pca = PCA(n_components=120)
X_train_pca = pca.fit_transform(X_train_scaled)
X_test_pca = pca.transform(X_test_scaled)

# Ուսուցանել ենք Classifier-ը
clf = LogisticRegression(max_iter=1100, solver='lbfgs', multi_class='multinomial')

start_time = time.time()
clf.fit(X_train_pca, y_train)
training_time = time.time() - start_time

# Գնահատել ենք արդյունքները
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

print(f"Ուսուցման ժամանակը: {training_time:.2f} վայրկյան")
print(f"Ճշտությունը: {accuracy:.4f}")
print(f"Սկզբնական չափ: {X_train.shape[1]}, PCA-ից հետո: {X_train_pca.shape[1]}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Ուսուցման ժամանակը: 152.78 վայրկյան
Ճշտությունը: 0.8400
Սկզբնական չափ: 784, PCA-ից հետո: 120


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [8]:
# Նույն գործողությունները կատարել ենք այստեղ առանց PCA կիրառելու

import numpy as np
import time
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.datasets import fashion_mnist

random.seed(53)
np.random.seed(53)
os.environ['PYTHONHASHSEED'] = '53'

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train = X_train.reshape(-1, 28*28)
X_test = X_test.reshape(-1, 28*28)


scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)


clf = LogisticRegression(max_iter=1100, solver='lbfgs', multi_class='multinomial')

start_time = time.time()
clf.fit(X_train_scaled, y_train)
training_time = time.time() - start_time

y_pred = clf.predict(X_test_scaled)
accuracy = accuracy_score(y_test, y_pred)

print(f"Ուսուցման ժամանակը առանց PCA-ի: {training_time:.2f} վայրկյան")
print(f"Ճշտությունը առանց PCA-ի: {accuracy:.4f}")
print(f"Մուտքային feature-ները: {X_train.shape[1]}")

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Ուսուցման ժամանակը առանց PCA-ի: 439.51 վայրկյան
Ճշտությունը առանց PCA-ի: 0.8347
Մուտքային feature-ները: 784


In [ ]:
# Ձեռքով իմպլեմենտացրել ենք PCA-ը և կիրառել կրկին Fashion MNIST դատասետի վրա
import numpy as np
import time
from tensorflow.keras.datasets import fashion_mnist
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Ներբեռնենք տվյալնորը
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()

X_train = X_train.reshape(-1, 28*28).astype(np.float32)
X_test = X_test.reshape(-1, 28*28).astype(np.float32)

# Նորմավորենք տվյալները
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0) + 1e-8
X_train_norm = (X_train - mean) / std
X_test_norm = (X_test - mean) / std

def pca(X, num_components):
    # Հանենք միջինը
    X_centered = X - np.mean(X, axis=0)

    # Հաշվենք կովարիացիոն մատրիցը
    covariance_matrix = np.cov(X_centered, rowvar=False)

   
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # Սորտավորենք սեփական վեկտորները
    sorted_idx = np.argsort(eigenvalues)[::-1]
    eigenvectors_sorted = eigenvectors[:, sorted_idx]

    # Ընտրում ենք առաջին k սեփական վեկտորները
    components = eigenvectors_sorted[:, :num_components]

    # Պրոյեկտում ենք կետերը
    X_reduced = np.dot(X_centered, components)

    return X_reduced, components


# PCA-ի կիրառումը
n_components = 120
X_train_pca, components = pca(X_train_norm, n_components)
X_test_pca = np.dot(X_test_norm - np.mean(X_train_norm, axis=0), components)


# Կլասիֆիկատորի ուսուցումը
clf = LogisticRegression(max_iter=1000, solver='lbfgs', multi_class='multinomial', random_state=42)

start = time.time()
clf.fit(X_train_pca, y_train)
train_time = time.time() - start


# Գնահատենք
y_pred = clf.predict(X_test_pca)
accuracy = accuracy_score(y_test, y_pred)

print(f"Ուսուցման ժամանակը: {train_time:.2f} վայրկյան")
print(f"Ճշտությունը: {accuracy:.4f}")
print(f"կրճատված չափողականություն: {X_train_pca.shape[1]}")


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:1247: FutureWarning: 'multi_class' was deprecated in version 1.5 and will be removed in 1.7. From then on, it will always use 'multinomial'. Leave it to its default value to avoid this warning.
  warnings.warn(


Ուսուցման ժամանակը: 137.95 վայրկյան
Ճշտությունը: 0.8404
կրճատված չափողականություն: 120


/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
